In [1]:
# User-defined variables
folder = "./good_model_inputs/";

In [2]:
using DataFrames, CSV, JuMP, CPLEX, Test

┌ Info: Recompiling stale cache file /home/emfreese/.julia/compiled/v1.1/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1184
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/emfreese/.julia/compiled/v1.1/JuMP/DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/emfreese/.julia/compiled/v1.1/CPLEX/QpT0w.ji for CPLEX [a076750e-1247-5638-91d2-ce28b192dca0]
└ @ Base loading.jl:1184


In [4]:
gen = CSV.read(folder * "inputs_gen_no-nuclear_all-generators_20k.csv")
load = CSV.read(folder * "inputs_load_no-nuclear.csv")
renCF = CSV.read(folder * "inputs_renewableCF.csv")
trans = CSV.read(folder * "inputs_trans_no-nuclear.csv")
;

ArgumentError: ArgumentError: "./good_model_inputs/inputs_gen_no-nuclear_all-generators_20k.csv" is not a valid file

In [5]:
regions = unique(load[:r])
tList = 5080:5080
nTime = length(tList)
nGen = size(gen)[1]
nTrans = size(trans)[1]
transLoss = 0.972;

UndefVarError: UndefVarError: load not defined

In [6]:
gen

UndefVarError: UndefVarError: gen not defined

In [5]:
m = Model(with_optimizer(CPLEX.Optimizer))

@variable(m, xgen[i=1:nGen, t=1:nTime] >= 0)
@variable(m, xtrans[j=1:nTrans, t=1:nTime] >= 0)


# Constrain generation
for idx = 1:nGen
    if gen[:FuelType][idx] == "Solar"
        @constraint(m, xgen[idx,:] - gen[:Capacity][idx] * renCF[tList, :solarCF] .<= 0)
    elseif gen[:FuelType][idx] == "Wind"
        @constraint(m, xgen[idx,:] - gen[:Capacity][idx] * renCF[tList, :windCF] .<= 0)
    elseif gen[:FuelType][idx] == "Nuclear"
        @constraint(m, xgen[idx,:] - gen[:Capacity][idx] * 0.95 .<= 0)
    else
        @constraint(m, xgen[idx,:] - gen[:Capacity][idx] .<= 0)
    end
end
            
# Constrain transmission
for idx = 1:nTrans
    @constraint(m, xtrans[idx,:] - trans[:transCap][idx] .<= 0)
end


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[5]:9
└ @ Core ./In[5]:9
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[5]:11
└ @ Core ./In[5]:11
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[5]:13
└ @ Core ./In[5]:13
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at macros.jl:390 [inlined]
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:390
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at macros.jl:390 [inlined]
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:390
┌ Warning: `getindex(df::DataFrame, col_ind::Colu

In [6]:
# Generation must match load
for reg in regions
    for t in 1:nTime
        tim = tList[t]
        ld = load[load[:r] .== reg, :demandLoad][tim] # regional load at that time
        @constraint(m, sum(xgen[i,t] for i=1:nGen if gen[:RegionName][i] == reg) +
                        sum(xtrans[j,t] for j=1:nTrans if trans[:r2][j] == reg) * transLoss -
                        sum(xtrans[j,t] for j=1:nTrans if trans[:r1][j] == reg) -
                        ld .>= -100) # + load
    end
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[6]:5
└ @ Core ./In[6]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at macros.jl:390 [inlined]
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:390
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at macros.jl:396 [inlined]
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:396
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at macros.jl:396
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:396


In [7]:
gen

,Column1,RegionName,FuelType,FuelCostTotal,PLCO2RTA,ORISCode,DIVISIONCE
,String,String,String,Float64,Float64⍰,String,String⍰
1,0,ERC_FRNT,NaturalGas,19.6867,395.82,55062,WSC-TX
2,1,ERC_FRNT,NaturalGas,19.6867,395.82,55062,WSC-TX
3,2,ERC_FRNT,NaturalGas,19.6867,395.82,55062,WSC-TX
4,3,ERC_FRNT,NaturalGas,19.6867,395.82,55062,WSC-TX
5,4,ERC_GWAY,NaturalGas,20.7378,397.927,55132,WSC-TX
6,5,ERC_GWAY,NaturalGas,20.7378,397.927,55132,WSC-TX
7,6,ERC_GWAY,NaturalGas,20.7378,397.927,55132,WSC-TX
8,7,ERC_GWAY,NaturalGas,20.7378,397.927,55132,WSC-TX
9,8,ERC_REST,Biomass,31.6,0.0,55708,WSC-TX


In [8]:
@objective(m, Min, sum(xgen[i,t] * gen[:FuelCostTotal][i] for i=1:nGen, t=1:nTime) +
                    sum(xtrans[j,t] * trans[:transCost][j] for j=1:nTrans, t=1:nTime))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at macros.jl:978
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:978
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at macros.jl:396
└ @ Core /home/emfreese/.julia/packages/JuMP/MsUSY/src/macros.jl:396


19.686689059561125 xgen[1,1] + 19.686689059561125 xgen[2,1] + 19.686689059561125 xgen[3,1] + 19.686689059561125 xgen[4,1] + 20.737758051724132 xgen[5,1] + 20.737758051724132 xgen[6,1] + 20.737758051724132 xgen[7,1] + 20.737758051724132 xgen[8,1] + 31.6 xgen[9,1] + 39.6 xgen[10,1] + 40.4 xgen[11,1] + 23.273160666666662 xgen[12,1] + 16.604289361702122 xgen[13,1] + 22.785 xgen[14,1] + 24.045 xgen[15,1] + 20.96049090909091 xgen[16,1] + 17.251794418181813 xgen[17,1] + 17.26027946363636 xgen[18,1] + 32.44307 xgen[19,1] + 19.34965765957446 xgen[20,1] + 28.95051033333334 xgen[21,1] + 28.916996333333337 xgen[22,1] + 17.553862036363633 xgen[23,1] + 19.63645902127659 xgen[24,1] + 19.683630297872334 xgen[25,1] + 19.713819914893612 xgen[26,1] + 25.305 xgen[27,1] + 19.732688425531908 xgen[28,1] + 26.355 xgen[29,1] + 20.68554821276595 xgen[30,1] + 20.71196412765957 xgen[31,1] + 31.00603566666667 xgen[32,1] + 31.00603566666667 xgen[33,1] + 30.927836333333335 xgen[34,1] + 31.704244000000006 xgen[35,1] 

In [9]:
JuMP.optimize!(m)

Parallel mode: deterministic, using up to 16 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 22405 rows and 14828 columns.
Reduced LP has 61 rows, 7577 columns, and 11237 nonzeros.
Presolve time = 0.05 sec. (16.96 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =       1398452.024867
Iteration:   106   Dual objective     =      15911792.897147

Dual simplex solved model.



In [10]:
print("Optimization size N = ", nGen+nTrans)

Optimization size N = 22405

In [11]:
if termination_status(m) == MOI.OPTIMAL
    gen = value.(xgen)
    genOut = convert(DataFrame, gen)
    trans = value.(xtrans)
    transOut = convert(DataFrame, trans)
    optimal_objective = objective_value(m)
    println("Optimal")
    println(optimal_objective)
elseif termination_status(m) == MOI.TIME_LIMIT && has_values(m)
    suboptimal_gen = value.(xgen)
    genOut = convert(DataFrame, suboptimal_gen)
    suboptimal_trans = value.(xtrans)
    transOut = convert(DataFrame, suboptimal_trans)
    suboptimal_objective = objective_value(m)
    println("Suboptimal")
    println(suboptimal_objective)
else
    error("The model was not solved correctly.")
end

Optimal
1.6051337669740181e7


In [329]:
genOut = convert(DataFrame, gen)

,x1
,Float64
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,100.0


In [ ]:
names(tst)

In [ ]:
CSV.write("testopt.csv", tst)